# Proyecto 2 – Modelos de clasificación supervisada para evaluar la calidad de un automovil.

**Integrantes del grupo:**
- Yenny A Gonzalez Melendez.
- Mateo A Duque Moya.
- Daniel Fuquene Linares.



In [2]:
# Instala el paquete ucimlrepo usando pip
!pip3 install ucimlrepo


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip


In [6]:
pip install ucimlrepo


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [7]:
# Usamos el importador de la base para no cargar descargar la información .

from ucimlrepo import fetch_ucirepo

# fetch dataset
car_evaluation = fetch_ucirepo(id=19)

# data (as pandas dataframes)
X = car_evaluation.data.features
y = car_evaluation.data.targets

# metadata
print(car_evaluation.metadata)

# variable information
print(car_evaluation.variables)





{'uci_id': 19, 'name': 'Car Evaluation', 'repository_url': 'https://archive.ics.uci.edu/dataset/19/car+evaluation', 'data_url': 'https://archive.ics.uci.edu/static/public/19/data.csv', 'abstract': 'Derived from simple hierarchical decision model, this database may be useful for testing constructive induction and structure discovery methods.', 'area': 'Other', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 1728, 'num_features': 6, 'feature_types': ['Categorical'], 'demographics': [], 'target_col': ['class'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 1988, 'last_updated': 'Thu Aug 10 2023', 'dataset_doi': '10.24432/C5JP48', 'creators': ['Marko Bohanec'], 'intro_paper': {'ID': 249, 'type': 'NATIVE', 'title': 'Knowledge acquisition and explanation for multi-attribute decision making', 'authors': 'M. Bohanec, V. Rajkovič', 'venue': '8th Intl Workshop on Expert Systems and their Applications, 

**Descripción del problema tomando en cuenta  las fuentes:**

La base de datos proporcionada tiene como finalidad ofrecer información valiosa para la evaluación de distintos vehículos, tomando en cuenta diversas características que influyen en la percepción de calidad por parte de los consumidores. A continuación, relacionaremos las variables relevantes junto con sus respectivas categorías, que serán fundamentales para el análisis y la evaluación posterior. Este proceso incluye la aplicación de técnicas analíticas, así como el entrenamiento de modelos de clasificación supervisada para determinar la calidad de los automóviles. Las variables a considerar son las siguientes:


*   **Costo de compra:** muy alto, alto, medio o bajo.
*   **Mantenimiento:** muy alto, alto, medio o bajo.
*   **Mantenimiento:** muy alto, alto, medio o bajo.
*   **Puertas:** 2, 3, 4, 5  he incluso más.
*   **Personas:** 2, 4, he incluso más.
*   **Maletero:** pequeño, medio, grande.
*   **Seguridad:** baja, media, alta.

Tomando en cuenta la información relacionada anteriormente, es necesario realizar una conversion total de la base a valores numericos par realizar correctamente el entrenamiento.















In [8]:
from ucimlrepo import fetch_ucirepo
import pandas as pd

# Dataset
car_evaluation = fetch_ucirepo(id=19)

# Data (as pandas dataframes)
X = car_evaluation.data.features
y = car_evaluation.data.targets

# Asigna X a df
df = X

# Obtener la información de las variables del dataset
variables_info = car_evaluation.variables

# Listas para almacenar los nombres de las variables numéricas y cualitativas
numericas = []
cualitativas = []

# Recorrer la información de las variables y clasificarlas
# Se itera sobre las filas del DataFrame variables_info
for index, row in variables_info.iterrows():
    # Se accede a los valores de las columnas usando el nombre de la columna
    if row['type'] == 'numeric':
        numericas.append(row['name'])
    else:  # Si no es numérica, se considera cualitativa
        cualitativas.append(row['name'])

# Tomar las columnas numéricas a tipo numérico (int o float)
for col in numericas:
    df[col] = pd.to_numeric(df[col])

# Campos Resumen
resumen = {
    'Número de variables': df.shape[1],
    'Número de registros': df.shape[0],
    'Variables numéricas': numericas,
    'Variables cualitativas': cualitativas,
}

# Imprimir.
print(resumen)

{'Número de variables': 6, 'Número de registros': 1728, 'Variables numéricas': [], 'Variables cualitativas': ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']}


**Resumen De La información:**


Después de llevar a cabo un análisis inicial de los datos, podemos concluir que contamos con un total de 6 variables clasificatorias y 1,728 registros. Estas variables abarcan una amplia gama de características que nos permiten identificar las preferencias generales en el estudio. Cabe destacar que todas las variables son de la naturaleza clasificatoria que desarrollaremos a continuación.

Es necesario adicionar la conversión que realizamos a las variables para su evaluación y entrenamiento como lo veremos a continuación en el print generado con la nueva tabla:

*   **Costo de compra:** muy alto = 4, alto = 3, medio = 2  y  bajo = 1.
*   **Mantenimiento:** muy alto = 4, alto = 3, medio = 2  y  bajo = 1.
*   **Puertas:** 2, 3, 4, 5, 5 o más = 6.
*   **Personas:** 2, 4, más = 5.
*   **Maletero:** pequeño = 1, medio = 2, grande = 3.
*   **Seguridad:** baja = 1, media = 2, alta = 3.

In [9]:
# Modificaciones realizadas (Diccionario)
conversion_dict = {
    'buying': {'vhigh': 4, 'high': 3, 'med': 2, 'low': 1},
    'maint': {'vhigh': 4, 'high': 3, 'med': 2, 'low': 1},
    'doors': {'2': 2, '3': 3, '4': 4, '5more': 6 },
    'persons': {'2': 2, '4': 4, 'more': 5},
    'lug_boot': {'small': 1, 'med': 2, 'big': 3},
    'safety': {'low': 1, 'med': 2, 'high': 3}
}


# Valores únicos de cada columna
for col in conversion_dict.keys():
    if col in df.columns:
        print(f"Valores únicos en '{col}': {df[col].unique()}")



# uso de la función replace
for col, mapping in conversion_dict.items():
    if col in df.columns:
        df[col] = df[col].replace(mapping)

# Pantallazo del DataFrame después de la conversión :)
print(df)




Valores únicos en 'buying': ['vhigh' 'high' 'med' 'low']
Valores únicos en 'maint': ['vhigh' 'high' 'med' 'low']
Valores únicos en 'doors': ['2' '3' '4' '5more']
Valores únicos en 'persons': ['2' '4' 'more']
Valores únicos en 'lug_boot': ['small' 'med' 'big']
Valores únicos en 'safety': ['low' 'med' 'high']
      buying  maint  doors  persons  lug_boot  safety
0          4      4      2        2         1       1
1          4      4      2        2         1       2
2          4      4      2        2         1       3
3          4      4      2        2         2       1
4          4      4      2        2         2       2
...      ...    ...    ...      ...       ...     ...
1723       1      1      6        5         2       2
1724       1      1      6        5         2       3
1725       1      1      6        5         3       1
1726       1      1      6        5         3       2
1727       1      1      6        5         3       3

[1728 rows x 6 columns]


/var/folders/13/1yqm_l0d1y1823t4h71nr_2m0000gn/T/ipykernel_47298/548952707.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].replace(mapping)


In [10]:
# Valores únicos de cada columna
for col in conversion_dict.keys():
    if col in df.columns:
        print(f"Valores únicos en '{col}': {df[col].unique()}")

Valores únicos en 'buying': [4 3 2 1]
Valores únicos en 'maint': [4 3 2 1]
Valores únicos en 'doors': [2 3 4 6]
Valores únicos en 'persons': [2 4 5]
Valores únicos en 'lug_boot': [1 2 3]
Valores únicos en 'safety': [1 2 3]


**División para entrenamiento:**

A continuación relacionamos la division de la variable class para determinar el entrenamiento 80% y testeo del 20%.


In [12]:
pip install scikit-learn


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 11.0 MB 7.3 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [13]:
from sklearn.model_selection import train_test_split

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)

# Print de los conjuntos de entrenamiento y prueba
print("Forma del conjunto de entrenamiento (X_train):", X_train.shape)
print("Forma del conjunto de prueba (X_test):", X_test.shape)
print("Forma del conjunto de entrenamiento (y_train):", y_train.shape)
print("Forma del conjunto de prueba (y_test):", y_test.shape)

Forma del conjunto de entrenamiento (X_train): (1382, 6)
Forma del conjunto de prueba (X_test): (346, 6)
Forma del conjunto de entrenamiento (y_train): (1382, 1)
Forma del conjunto de prueba (y_test): (346, 1)


**Modelos De Aprendisaje Que Implementaremos:**


*  Árbol de decisión, DecisionTreeClassifier
*  Bosque aleatorio, RandomForestClassifier
*  Regresión logística, LogisticRegression
*  K-vecinos más cercanos, KNeighborsClassifier
*  Máquina de vectores de soporte", SVC
*  Naive Bayes, GaussianNB



In [14]:
from ucimlrepo import fetch_ucirepo
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

# Dataset
car_evaluation = fetch_ucirepo(id=19)

# Data (as pandas dataframes)
X = car_evaluation.data.features
y = car_evaluation.data.targets

# Obtener la información de las variables del dataset
variables_info = car_evaluation.variables

# Listas para almacenar los nombres de las variables numéricas y cualitativas
numericas = []
cualitativas = []

# Recorrer la información de las variables y clasificarlas
for index, row in variables_info.iterrows():
    if row['type'] == 'numeric':
        numericas.append(row['name'])
    else:
        cualitativas.append(row['name'])

# Excluir la columna 'class' de la lista cualitativas
cualitativas = [col for col in cualitativas if col != 'class']

# Convertir variables cualitativas a numéricas usando Label Encoding
for col in cualitativas:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Lista de modelos para evaluar
modelos = [
    ("Árbol de decisión", DecisionTreeClassifier()),
    ("Bosque aleatorio", RandomForestClassifier()),
    ("Regresión logística", LogisticRegression(max_iter=1000)),
    ("K-vecinos más cercanos", KNeighborsClassifier()),
    ("Máquina de vectores de soporte", SVC()),
    ("Naive Bayes", GaussianNB())
]

# Iterar sobre los modelos y generar los reportes de clasificación
for nombre_modelo, modelo in modelos:
  # Entrenar el modelo
  modelo.fit(X_train, y_train.values.ravel()) # Ajustar para usar .values.ravel()

  # Realizar predicciones en el conjunto de prueba
  y_pred = modelo.predict(X_test)

  # Generar el reporte de clasificación
  print(f"Reporte de clasificación para {nombre_modelo}:\n")
  print(classification_report(y_test, y_pred))
  print("\n" + "-"*50 + "\n")

Reporte de clasificación para Árbol de decisión:

              precision    recall  f1-score   support

         acc       0.97      0.92      0.94        83
        good       0.56      0.91      0.69        11
       unacc       1.00      1.00      1.00       235
       vgood       1.00      0.82      0.90        17

    accuracy                           0.97       346
   macro avg       0.88      0.91      0.88       346
weighted avg       0.98      0.97      0.97       346


--------------------------------------------------

Reporte de clasificación para Bosque aleatorio:

              precision    recall  f1-score   support

         acc       0.99      0.88      0.93        83
        good       0.56      0.91      0.69        11
       unacc       0.99      1.00      1.00       235
       vgood       0.94      0.94      0.94        17

    accuracy                           0.97       346
   macro avg       0.87      0.93      0.89       346
weighted avg       0.97      0.97

/var/folders/13/1yqm_l0d1y1823t4h71nr_2m0000gn/T/ipykernel_47298/138785377.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = le.fit_transform(X[col])
/var/folders/13/1yqm_l0d1y1823t4h71nr_2m0000gn/T/ipykernel_47298/138785377.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = le.fit_transform(X[col])
/var/folders/13/1yqm_l0d1y1823t4h71nr_2m0000gn/T/ipykernel_47298/138785377.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

**Resumen de los modelos aplicados**

**Árbol de Decisión y Bosque Aleatorio:**

Buen desempeño general: Ambos modelos muestran un buen desempeño, especialmente en la clase "unacc".
F1-score alto: El F1-score es alto para ambas clases, lo que indica un buen equilibrio entre precisión y recall.
Bosque Aleatorio ligeramente mejor: El Bosque Aleatorio parece tener un ligero borde sobre el Árbol de Decisión en términos de precisión y F1-score promedio. Esto es común, ya que los Bosques Aleatorios son un ensamblaje de múltiples árboles de decisión, lo que reduce el sobreajuste y mejora la precisión.

**Regresión Logística:**

Desempeño pobre: La Regresión Logística tiene un desempeño significativamente peor que los otros modelos, especialmente en las clases "bueno" y "vgood".
Sesgo hacia la clase mayoritaria: El modelo parece estar sesgado hacia la clase mayoritaria ("unacc"), lo que podría deberse a un desbalance en los datos.

**K-vecinos más cercanos y Máquina de Vectores de Soporte:**

Desempeño mixto: Estos modelos tienen un desempeño variable dependiendo de la clase.
Sensibilidad a los hiperparámetros: El desempeño de estos modelos puede ser muy sensible a la elección de los hiperparámetros (como el número de vecinos en KNN o el kernel en SVM).

**Naive Bayes:**

Peor desempeño: Naive Bayes tiene el peor desempeño general de todos los modelos.
Asunción de independencia: El supuesto de independencia condicional de Naive Bayes puede no ser válido en muchos casos reales, lo que puede afectar su precisión.


**PUNTO 11**

Basándome en el F1-score promedio y la consistencia en el desempeño a través de las diferentes clases, el Bosque Aleatorio parece ser el mejor candidato. Su capacidad de crear múltiples árboles de decisión y combinar sus resultados lo hace más robusto y menos propenso al sobreajuste.

el Bosque Aleatorio es un buen punto de partida, pero hay margen para mejorar el rendimiento del modelo. Ajustando los hiperparámetros, abordando el desequilibrio de clases y explorando otras técnicas, es posible obtener resultados aún mejores.